In [1]:
import numpy as np
import cv2
import glob

from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
name= 'New_runs_with_car'
file_list=np.array(glob.glob(f"runs/{name}/**/discrete_histories.npz"))
file_list.shape

(100,)

In [3]:
np.random.shuffle(file_list)
train=file_list[:int(file_list.shape[0]*.8)]
test = file_list[int(file_list.shape[0]*.8):]

In [4]:
print("train ", train.shape)
print("test ", test.shape)
np.savez(f"runs/{name}/Dataset_div", train_files=train, test_files=test)

train  (80,)
test  (20,)


In [5]:
test_targets=[]
for file in test:
    data = np.load(file)
    test_targets+=data["input_history"].tolist()
test_targets = np.array(test_targets).flatten() 
test_targets.shape

(18576,)

In [48]:
train_targets=[]
for file in train:
    data = np.load(file)
    train_targets+=data["input_history"].tolist()
train_targets = np.array(train_targets).flatten() 
train_targets.shape

(75632,)

In [ ]:
fig = plt.figure(figsize=(15,5))
X = np.arange(4)

plt.title(f"Dataset target distribution for: \nTraining dataset {train_targets.shape[0]} examples, from {train.shape[0]} runs \nTest dataset {test_targets.shape[0]} examples, from {test.shape[0]} runs ")
plt.bar(X + 0.00,np.unique(test_targets, return_counts=True)[1], color = 'b', width = 0.25)
plt.bar(X + 0.25,np.unique(train_targets, return_counts=True)[1],color = 'g', width = 0.25 )

directions = ["Left", 'Right', 'Accelerate', "Brake" ]
plt.legend(labels=['Test', 'Train'])
plt.xticks(X, directions)
plt.savefig(f"Plots/dataset_statistics_{name}.png", dpi=fig.dpi, bbox_inches='tight', pad_inches=0.5)
plt.show()


In [ ]:
draw_car = False
if draw_car:
    car = np.load("car_positions.npz")
    car_index_red=np.where(car['frame'][ :, :, 0] == 204)
    car_index_black = np.where(car['frame'][ :256, :, 0] == 0)

input_history_train = []

data = np.load(train[0])
#input_history_train.append(data["input_history"]) # 
input_history_train+=data["input_history"].tolist()

aux_frame = np.copy(data["car_frames"])
if draw_car:
    aux_frame[:,car_index_red[0],car_index_red[1],: ] = [204, 0, 0]
    aux_frame[:,car_index_black[0],car_index_black[1],: ] = [0, 0, 0]
    
aux_frame = np.transpose(aux_frame, axes=[0, 3, 1, 2])
train_frames = (2 * (aux_frame / 255)) - 1

for file in train[1:]:
    data = np.load(file)
    #input_history_train.append(data["input_history"]) # 
    input_history_train+=data["input_history"].tolist()
    aux_frame = np.copy(data["car_frames"])
    if draw_car:
        aux_frame[:,car_index_red[0],car_index_red[1],: ] = [204, 0, 0]
        aux_frame[:,car_index_black[0],car_index_black[1],: ] = [0, 0, 0]
    aux_frame = np.transpose(aux_frame, axes=[0, 3, 1, 2])
    aux_frame = (2 * (aux_frame / 255)) - 1
    train_frames = np.vstack((train_frames,aux_frame ))

input_history_train = np.array(input_history_train).flatten()


In [ ]:
input_history = []

data = np.load(test[0])
#input_history.append(data["input_history"]) # 
input_history+=data["input_history"].tolist()

aux_frame = np.copy(data["car_frames"])
if draw_car:
    aux_frame[:,car_index_red[0],car_index_red[1],: ] = [204, 0, 0]
    aux_frame[:,car_index_black[0],car_index_black[1],: ] = [0, 0, 0]
aux_frame = np.transpose(aux_frame, axes=[0, 3, 1, 2])
test_frames = (2 * (aux_frame / 255)) - 1

for file in test[1:3]:
    data = np.load(file)
    #input_history.append(data["input_history"]) # 
    input_history+=data["input_history"].tolist()
    
    aux_frame = np.copy(data["car_frames"])
    if draw_car:
        aux_frame[:,car_index_red[0],car_index_red[1],: ] = [204, 0, 0]
        aux_frame[:,car_index_black[0],car_index_black[1],: ] = [0, 0, 0]
    aux_frame = np.transpose(aux_frame, axes=[0, 3, 1, 2])
    aux_frame = (2 * (aux_frame / 255)) - 1
    test_frames = np.vstack((test_frames, aux_frame))

input_history = np.array(input_history).flatten()

In [ ]:
np.savez_compressed(f"runs/{name}/Dataset_complete", train_files=train, test_files=test, input_history_train=input_history_train, train_frames=train_frames, input_history_test=input_history, test_frames=test_frames)

In [ ]:
# data = np.load("./runs/Dataset_complete.npz")
#
#
# data.files
#
#
# input_history_train=data['input_history_train']
# train_frames=data['train_frames']
# input_history=data['input_history_test']
# test_frames=data['test_frames']



# balance

In [ ]:
indices =np.unique(input_history_train, return_counts=True)[1]
indices.sort()
indices[::-1]

In [ ]:
equalized_commands = np.append(input_history_train[np.where(input_history_train==1)][:int(indices[::-1][2]*1.1)],input_history_train[np.where(input_history_train==2)][:int(indices[::-1][2]*1.1)])

equalized_commands = np.append(equalized_commands,input_history_train[np.where(input_history_train==4)])
equalized_commands = np.append(equalized_commands,input_history_train[np.where(input_history_train==3)][:int(indices[::-1][2]*1.1)])

In [ ]:
equalized_frames = np.vstack((train_frames[np.where(input_history_train==1)][:int(indices[::-1][2]*1.1)],train_frames[np.where(input_history_train==2)][:int(indices[::-1][2]*1.1)]))

equalized_frames = np.vstack((equalized_frames,train_frames[np.where(input_history_train==4)]))
equalized_frames = np.vstack((equalized_frames,train_frames[np.where(input_history_train==3)][:int(indices[::-1][2]*1.1)]))

In [ ]:
fig = plt.figure(figsize=(15,5))
X = np.arange(4)

plt.title(f"Dataset target distribution for: \nTraining dataset {equalized_commands.shape[0]} examples")
plt.bar(X + 0.00,np.unique(equalized_commands, return_counts=True)[1], color = 'b', width = 0.25)
plt.bar(X + 0.25,np.unique(input_history_train, return_counts=True)[1],color = 'g', width = 0.25 )

directions = ["Left", 'Right', 'Accelerate', "Brake" ]
plt.legend(labels=['Balanced', 'Unbalanced'])
plt.xticks(X, directions)
plt.savefig(f"Plots/dataset_statistics_balanced_{name}.png", dpi=fig.dpi, bbox_inches='tight', pad_inches=0.5)
plt.show()

In [ ]:
equalized_frames.shape

In [ ]:
equalized_commands.shape

In [ ]:
equalized_frames.shape,equalized_frames.min(), equalized_frames.max()

In [ ]:
np.savez_compressed(f"runs/{name}/Dataset_equalized", input_history_train=equalized_commands, train_frames=equalized_frames, input_history_test=input_history, test_frames=test_frames)

In [ ]:
Staaaaaaaap here

# Convert multiple frames and back

In [ ]:
data = np.load(train[0])
frame = data["car_frames"][0:2]

In [ ]:
frame.shape,frame.min(), frame.max(),frame.dtype

In [ ]:
plt.imshow(frame[0])

In [ ]:
aux_frame = np.transpose(frame, axes=[0, 3, 1, 2])
aux_frame = (2 * (aux_frame / 255)) - 1

In [ ]:
aux_frame.shape,aux_frame.min(), aux_frame.max(),aux_frame.dtype

In [ ]:
conv_frame = np.transpose(aux_frame, axes=[0,2,3,1])

In [ ]:
conv_frame =(((conv_frame + 1)/2)*255).astype(np.uint8)

In [ ]:
conv_frame.shape,conv_frame.min(), conv_frame.max(), conv_frame.dtype

In [ ]:
plt.imshow(conv_frame[0])

# Convert one frame and back

In [ ]:
data = np.load(train[0])
frame = data["car_frames"][463]

In [ ]:
frame.shape,frame.min(), frame.max(),frame.dtype

In [ ]:
plt.imshow(frame)

In [ ]:
aux_frame = np.transpose(frame, axes=[2, 0, 1])
aux_frame = (2 * (aux_frame / 255)) - 1

In [ ]:
aux_frame.shape,aux_frame.min(), aux_frame.max(),aux_frame.dtype

In [ ]:
conv_frame = np.transpose(train_frames[0], axes=[1,2,0])

In [ ]:
conv_frame = (((conv_frame + 1)/2)*255).astype(np.uint8)

In [ ]:
conv_frame.shape,conv_frame.min(), conv_frame.max(), conv_frame.dtype

In [ ]:
plt.imshow(conv_frame)

# Add car

In [ ]:
aux_frame = np.copy(data["car_frames"])

In [ ]:
plt.imshow(aux_frame[0])

In [ ]:
aux_frame.shape

In [ ]:
plt.imshow(car['frame'])

In [ ]:
aux_frame[:,car_index_red[0],car_index_red[1],: ] = [204, 0, 0]
aux_frame[:,car_index_black[0],car_index_black[1],: ] = [0, 0, 0]

In [ ]:
plt.imshow(aux_frame[0])